# Measure of consensus

In [ ]:
output_path
input_path

In [2]:
import pandas as pd

In [ ]:
df_policy = pd.read_csv("C:/Users/easycash/Mon Drive/Thèse/1_Systematic mapping/6_structural_topic_model/5_final_db/3_policy_and_factors_clustered.csv"  )
df_init = pd.read_csv("C:/Users/easycash/Mon Drive/Thèse/1_Systematic mapping/6_structural_topic_model/3_exit/extract_policies_ML_concat.csv")

In [ ]:
# Convert 'index' to numeric, coercing errors to NaN
df_policy['row_index'] = pd.to_numeric(df_policy['row_index'], errors='coerce')

# Handle NaN values (e.g., drop rows with NaN)
df_policy = df_policy.dropna(subset=['row_index'])
df_policy = df_policy.dropna(subset=['matched_cluster_factor'])

# Convert to integer
df_policy['row_index'] = df_policy['row_index'].astype(int)

data = df_policy.set_index('row_index').join(df_init.set_index('Index')['abstract'], how='left').reset_index()

## Prevalence of policies in abstract Using Proportional Sentence Match

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util

In [ ]:
df = data

In [ ]:
# Load pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# Function to compute proportional sentence match
def compute_proportional_sentence_match(df, threshold=0.8):
    results = []
    
    for index, row in df.iterrows():
        # Encode the abstract and POLICY sentences
        abstract_embedding = model.encode(row['abstract'], convert_to_tensor=True)
        policy_sentences = row['POLICY'].split(".")  # Split POLICY into individual sentences
        policy_embeddings = model.encode(policy_sentences, convert_to_tensor=True)
        
        # Compute cosine similarity
        similarities = util.cos_sim(policy_embeddings, abstract_embedding).numpy()
        
        # Metrics
        #relevant_sentences = (similarities > threshold).sum()
        #proportion_relevant = relevant_sentences / len(policy_sentences)
        average_similarity = similarities.mean()
        
        # Store results
        results.append({
            'abstract': row['abstract'],
            'POLICY': row['POLICY'],
            #'proportion_relevant': proportion_relevant,
            'average_similarity': average_similarity
        })
    
    return pd.DataFrame(results)

In [ ]:
# Apply the function
results_df = compute_proportional_sentence_match(df)

In [ ]:
df['average_policy_similarity'] = results_df['average_similarity']

In [ ]:
df.loc[df['CORRELATION']=='increasing','CORRELATION_num'] = 1
df.loc[df['CORRELATION']=='decreasing','CORRELATION_num'] = -1
df.loc[df['CORRELATION']=='neutral','CORRELATION_num'] = 0

# Normalize policy_similarity per matched_cluster
df['policy_similarity_normalized_by_cluster'] = df.groupby('matched_cluster')['average_policy_similarity'] \
    .transform(lambda x: (x - x.min()) / (x.max() - x.min()))

# Normalize policy_similarity per matched_cluster_factor
df['policy_similarity_normalized_by_factor'] = df.groupby('matched_cluster_factor')['average_policy_similarity'] \
    .transform(lambda x: (x - x.min()) / (x.max() - x.min()))

# If you want global normalization
df['policy_similarity_normalized_global'] = (df['average_policy_similarity'] - df['average_policy_similarity'].min()) / \
                                            (df['average_policy_similarity'].max() - df['average_policy_similarity'].min())

df['correlation_prod_normalized_by_cluster'] = df['policy_similarity_normalized_by_cluster']*df['CORRELATION_num']
df['correlation_prod_normalized_by_factor'] = df['policy_similarity_normalized_by_factor']*df['CORRELATION_num']
df['correlation_prod_normalized_global'] = df['policy_similarity_normalized_global']*df['CORRELATION_num']

## Export calculus

In [ ]:
cluster_df = pd.read_csv("C:/Users/easycash/Mon Drive/Thèse/1_Systematic mapping/6_structural_topic_model/4_clustering/2_modif_cluster.csv", sep=";" )
cluster_df_factor = pd.read_csv("C:/Users/easycash/Mon Drive/Thèse/1_Systematic mapping/6_structural_topic_model/4_clustering/1_outcomes/2_modif_cluster.csv", sep=";" )

In [ ]:
updated_df=df.dropna(subset='matched_cluster')
updated_df = pd.merge(updated_df, cluster_df[['Cluster Name','Agg Cluster']], how= 'left', left_on= 'matched_cluster', right_on= 'Cluster Name')
grouped = updated_df.groupby(["Agg Cluster","Cluster Name"])["matched_cluster"].count()

updated_df = pd.merge(updated_df, cluster_df_factor[['Cluster Name','Agg Cluster','Corr Sign']], how= 'left', left_on= 'matched_cluster_factor', right_on= 'Cluster Name',suffixes=(False, '_factor'))

In [ ]:
# Save the updated dataset
output_path = "C:/Users/easycash/Mon Drive/Thèse/1_Systematic mapping/6_structural_topic_model/5_final_db/4_policy_and_factors_clustered_prevalence_normalized.csv" 

# Update with your desired output path
updated_df.to_csv(output_path, index=False)